# Compute projections
Execute `project_roi.sh` shell script to start Slurm jobs to compute tile projections:

`./project_roi.sh becalia_rabies_barseq/BRYC65.1d/brain/slide_008/section_02/ 1 8 8 7 round`

`./project_roi.sh $PATH $ROI $XTILES $YTILES $ROUNDS $PREFIX`

# Import requirements

In [ ]:
import iss_preprocess as iss
import numpy as np
import matplotlib.pyplot as plt
from flexiznam.config import PARAMETERS
from pathlib import Path
from itertools import cycle
from matplotlib.animation import FuncAnimation
%matplotlib ipympl


# Estimate registration for rounds and channels

In [ ]:
ops = { 
    'ref_tile': (1,2,7), 
    'ref_ch': 0,
    'ref_round': 0,
    'filter_r': (2, 4),
    'detection_threshold': 40,
    'isolation_threshold': 30,
    'codebook': 'codebook_83gene_pool.csv',
    'projection': 'proj',
    'omp_threshold': 0.2,
    'omp_max_genes': 12,
    'spot_threshold': 0.15,
    'spot_rho': 2
    }
data_path = 'becalia_rabies_barseq/BRYC65.1d/brain/slide_008/section_02/'
processed_path = Path(PARAMETERS['data_root']['processed'])

In [ ]:
stack = iss.pipeline.load_processed_tile(data_path, ops['ref_tile'])
tforms = iss.reg.register_channels_and_rounds(
    stack, ref_ch=ops['ref_ch'], ref_round=ops['ref_round']
)
save_path = processed_path / data_path / 'tforms.npy'
np.save(save_path, tforms, allow_pickle=True)
save_path = processed_path / data_path / 'ops.npy'
np.save(save_path, ops, allow_pickle=True)

# Prepare parameters for OMP

In [ ]:
stack, bad_pixels = iss.pipeline.load_and_register_tile(
    data_path, ops['ref_tile'], filter_r=ops['filter_r'], suffix=ops['projection']
)
gene_dict, gene_names, norm_shift = iss.pipeline.setup_omp(
    stack, 
    detection_threshold=ops['detection_threshold'], 
    isolation_threshold=ops['isolation_threshold'],
    codebook_name=ops['codebook']
)
save_path = processed_path / data_path / 'gene_dict.npz'
np.savez(save_path, gene_dict=gene_dict, gene_names=gene_names, norm_shift=norm_shift)

# Re-compute average spot image

Run OMP on the reference tile and detect spots to generate a new spot sign image.

Inspect OMP output to check parameters.

In [ ]:
g, b, r = iss.call.run_omp(
    stack,
    gene_dict,
    tol=ops['omp_threshold'],
    weighted=True,
    refit_background=True,
    alpha=200.,
    norm_shift=norm_shift,
    max_comp=12
)

spot_sign_image = iss.call.get_spot_shape(g, spot_xy=7, neighbor_filter_size=9, neighbor_threshold=15)
spot_sign_image = iss.call.apply_symmetry(spot_sign_image)
save_path = processed_path / data_path / 'spot_sign_image.npy'
np.save(save_path, spot_sign_image)
plt.imshow(spot_sign_image, cmap='bwr', vmin=-1, vmax=1)

plt.figure(figsize=(10,40))
for i in range(len(gene_names)):
    plt.subplot(20,5,i+1)
    plt.imshow(g[801:1100, 801:1100, i], cmap='bwr', vmin=-1, vmax=1)
    plt.title(gene_names[i])
plt.tight_layout()

In [ ]:
nrounds = stack.shape[2]
channel_colors = [ [1,0,0], [0,1,0], [0,1,1], [1,0,1]]
def round_image(iround):
    vmax = 50
    return iss.vis.to_rgb(
        stack[801:1100, 801:1100, iround, :], 
        channel_colors, 
        vmin=np.array([0,0,0,0]),
        vmax=np.array([1,1,1,1]) * vmax)

fig = plt.figure(figsize=(10,2))
for iround in range(nrounds):
    plt.subplot(1,7,iround+1)
    plt.imshow(round_image(iround))
plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(10,5))
ax1 = plt.subplot(1,2,1)
im = ax1.imshow(round_image(0))
plt.xlim([0,300])
plt.ylim([0,300])

def animate(frame_num):
    im.set_data(round_image(frame_num))
    return im

anim = FuncAnimation(fig, animate, frames=nrounds, interval=1000)
plt.show()
spot_sign_threshold = 0.15
spot_sign_image[np.abs(spot_sign_image) < spot_sign_threshold] = 0
gene_spots = iss.call.find_gene_spots(g[801:1100, 801:1100,:], spot_sign_image, 
        rho=ops['spot_rho'], omp_score_threshold=ops['spot_threshold'])
ax2 = plt.subplot(1,2,2)
for gene in gene_spots:
        ax2.plot(gene['x'], gene['y'], '.')
plt.xlim([0,300])
plt.ylim([0,300])

ax2.set_facecolor('black')




# Run OMP
Execute `extract_roi.sh` shell script to start Slurm jobs to run OMP on each tile:

`./extract_roi.sh becalia_rabies_barseq/BRYC65.1d/brain/slide_008/section_02/ 1 8 8`



# Merge OMP output

In [ ]:
shift_right, shift_down, tile_shape = iss.pipeline.register_adjacent_tiles(
    data_path, ref_coors=ops['ref_tile']
)
spots = iss.pipeline.merge_roi_spots(data_path, shift_right, shift_down, tile_shape, (9,9))
save_path = Path(PARAMETERS['data_root']['processed']) / data_path / 'gene_spots.pkl' 
spots.to_pickle(save_path)

In [ ]:
plt.style.use("default")

colors = cycle([ 'deepskyblue', 'aquamarine', 'orangered', 'violet', 'forestgreen', 'darkorange'])
markers = cycle('ov^<>spPXD*')
plt.figure(figsize=(12,80))

gene_names = spots['gene'].unique()

for igene, gene in enumerate(gene_names):
    plt.subplot(30,3,igene+1)
    plt.title(gene)
    plt.gca().set_aspect('equal', 'box')

    plt.plot(
        spots[spots['gene'] == gene]['x'],
        spots[spots['gene'] == gene]['y'],        
        '.',
        alpha=0.5,
        markersize=0.5
    )
    plt.axis('off')
    plt.gca().invert_yaxis()
plt.tight_layout()


# Generate an overview image and segment cells

In [ ]:
tile_origins, tile_centers = iss.pipeline.calculate_tile_positions(shift_right, shift_down, tile_shape, (9,9))

stitched_stack = iss.pipeline.stitch_tiles(data_path, 'round_01_1', tile_origins, tile_shape)

from cellpose.models import CellposeModel
model = CellposeModel(gpu=False, model_type='cyto', net_avg=True)
masks, flows, styles = model.eval(
    stitched_stack,
    rescale=.55,
    channels=[0,0],
    flow_threshold=0.4,
    tile=True
)
np.save(processed_path / data_path / 'masks_1.npy', masks)